## Create an example agent

In [1]:
"""An example agent."""

import random
from datetime import datetime
from dateutil import parser
import spacy
from rdflib import Namespace, URIRef, Literal, XSD
from humemai.rdflib import Humemai
from typing import Optional, List, Tuple, Dict

import logging

logging.getLogger("humemai").setLevel(logging.WARNING)


# Define the custom namespace for the ontology
humemai = Namespace("https://humem.ai/ontology#")


class Agent:
    def __init__(self):
        """
        Initialize the Agent with its memory system and NLP model.
        """
        self.memory = Humemai()
        self.nlp = spacy.load("en_core_web_sm")

    def process_input(self, sentence: str) -> None:
        """
        Process the input sentence, extract triples, location, and time, and add them to
        short-term memory. Afterward, check the working memory and decide which memories
        should be moved to long-term memory.

        Args:
            sentence (str): The natural language sentence to process.
        """
        doc = self.nlp(sentence)

        # Extract entities and relations
        entities = [(ent.text, ent.label_) for ent in doc.ents]
        print("\nEntities found:", entities)

        triples: List[Tuple[URIRef, URIRef, URIRef]] = []
        location: Optional[str] = None
        time: Optional[str] = None

        # Function to find the nearest verb to an entity
        def find_nearest_verb(token) -> Optional[str]:
            for ancestor in token.ancestors:
                if ancestor.pos_ == "VERB":
                    return ancestor.lemma_  # Use lemma of the verb
            return None

        # Extract triples (subject, predicate, object)
        for i, token in enumerate(doc):
            if token.ent_type_ in ("PERSON", "ORG", "GPE"):
                nearest_verb = find_nearest_verb(token)
                for j in range(i + 1, len(doc)):
                    next_token = doc[j]
                    if next_token.ent_type_ in ("PERSON", "ORG", "GPE"):
                        if nearest_verb:
                            triples.append(
                                (
                                    URIRef(f"https://example.org/{token.text}"),
                                    URIRef(f"https://example.org/{nearest_verb}"),
                                    URIRef(f"https://example.org/{next_token.text}"),
                                )
                            )
                        break

        # Extract location and time as qualifiers
        for ent in doc.ents:
            if ent.label_ == "GPE":
                location = ent.text
            elif ent.label_ == "DATE":
                try:
                    parsed_date = parser.parse(ent.text)
                    time = parsed_date.isoformat()
                except (ValueError, OverflowError):
                    time = None

        print("Extracted triples:", triples)
        print("Location:", location)
        print("Time:", time)

        # Add the extracted triples, location, and time to short-term memory
        self.add_to_short_term_memory(triples, location, time)

        # After adding to short-term memory, check if we should move any memories to long-term
        self.evaluate_and_move_memories()

    def add_to_short_term_memory(
        self,
        triples: List[Tuple[URIRef, URIRef, URIRef]],
        location: Optional[str] = None,
        time: Optional[str] = None,
    ) -> None:
        """
        Add triples into short-term memory along with location and time qualifiers

        Args:
            triples (list): List of triples (subject, predicate, object).
            location (str | None): Location qualifier.
            time (str | None): Time qualifier in ISO format.
        """
        for triple in triples:
            subj, pred, obj = triple

            qualifiers: Dict[URIRef, Literal] = {}

            # Add location and time if available
            if location:
                qualifiers[humemai.location] = Literal(location)
            if time:
                qualifiers[humemai.currentTime] = Literal(time, datatype=XSD.dateTime)

            self.memory.add_short_term_memory(
                [(subj, pred, obj)], qualifiers=qualifiers
            )

    def evaluate_and_move_memories(self) -> None:
        """
        Evaluate the current working memory (short-term and relevant long-term memories)
        and randomly decide which short-term memories should be moved to long-term memory.
        """
        # Get a random trigger node from the long-term memory if there are any
        long_term_memories = list(self.memory.iterate_memories("long_term"))

        # If there are long-term memories, randomly select a trigger node and hops
        if long_term_memories:
            random_memory = random.choice(long_term_memories)
            trigger_node = random.choice([random_memory[0], random_memory[2]])
            hops = random.randint(1, 3)  # Random hops from 1 to 3
            print(f"\nSelected trigger node: {trigger_node} with {hops} hops")
            include_all_long_term = False
        else:
            # No long-term memories, retrieve all memories
            trigger_node = None
            hops = 0
            include_all_long_term = True
            print("\nNo long-term memories available. Retrieving all memories.")

        # Get the current working memory
        working_memory = self.memory.get_working_memory(
            trigger_node=trigger_node,
            hops=hops,
            include_all_long_term=include_all_long_term,
        )

        # Iterate over short-term memories in the working memory
        for subj, pred, obj, qualifiers in working_memory.iterate_memories(
            "short_term"
        ):
            memory_id = qualifiers.get(humemai.memoryID)

            # Check if memory_id exists
            if memory_id is None:
                print(f"Skipping memory with no memoryID: {subj}, {pred}, {obj}")
                continue  # Skip memories without a memoryID

            # Randomly decide the memory type
            memory_type = random.choice(["episodic", "semantic"])

            if memory_type == "episodic":
                # Randomly assign emotion and event
                emotion = random.choice([None, "happy", "excited", "curious"])
                event = random.choice([None, "AI Conference", "Meeting", "Travel"])
                print(
                    f"Moving memory to episodic long-term with emotion: {emotion}, event: {event}"
                )
                self.move_memory_to_long_term(
                    memory_id=int(memory_id),
                    memory_type="episodic",
                    emotion=emotion,
                    event=event,
                )

            elif memory_type == "semantic":
                # Randomly assign strength and derivedFrom
                strength = random.randint(1, 10)  # Strength between 1 and 10
                derivedFrom = random.choice(
                    [None, "Observation", "Conversation", "Research"]
                )
                print(
                    f"Moving memory to semantic long-term with strength: {strength}, derivedFrom: {derivedFrom}"
                )
                self.move_memory_to_long_term(
                    memory_id=int(memory_id),
                    memory_type="semantic",
                    strength=strength,
                    derivedFrom=derivedFrom,
                )

    def move_memory_to_long_term(
        self,
        memory_id: int,
        memory_type: str,
        emotion: Optional[str] = None,
        strength: Optional[int] = None,
        derivedFrom: Optional[str] = None,
        event: Optional[str] = None,
    ) -> None:
        """
        Move the specified short-term memory to long-term memory.
        """
        # Retrieve the memory to modify
        memory = self.memory.get_memory_by_id(Literal(memory_id, datatype=XSD.integer))

        # If it's an episodic memory, we need to ensure 'currentTime' is removed and 'eventTime' is added
        if memory_type == "episodic":
            qualifiers = memory["qualifiers"]

            # Remove currentTime, which is not allowed in episodic memories
            if humemai.currentTime in qualifiers:
                del qualifiers[humemai.currentTime]

            # Add the required eventTime qualifier
            qualifiers[humemai.eventTime] = Literal(
                datetime.now().isoformat(), datatype=XSD.dateTime
            )

            # Optionally add emotion and event if provided
            if emotion:
                qualifiers[humemai.emotion] = Literal(emotion)
            if event:
                qualifiers[humemai.event] = Literal(event)

            # Move to long-term episodic memory
            self.memory.add_episodic_memory(
                [(memory["subject"], memory["predicate"], memory["object"])], qualifiers
            )

        elif memory_type == "semantic":
            qualifiers = memory["qualifiers"]

            # Remove disallowed qualifiers for semantic memories
            disallowed_qualifiers = [
                humemai.location,
                humemai.event,
                humemai.emotion,
                humemai.currentTime,
            ]
            for disallowed in disallowed_qualifiers:
                if disallowed in qualifiers:
                    del qualifiers[disallowed]

            # Add knownSince for semantic memories
            qualifiers[humemai.knownSince] = Literal(
                datetime.now().isoformat(), datatype=XSD.dateTime
            )

            # Optionally add strength and derivedFrom
            if strength is not None:
                qualifiers[humemai.strength] = Literal(strength, datatype=XSD.integer)
            if derivedFrom:
                qualifiers[humemai.derivedFrom] = Literal(derivedFrom)

            # Move to long-term semantic memory
            self.memory.add_semantic_memory(
                [(memory["subject"], memory["predicate"], memory["object"])], qualifiers
            )

        # Remove the original short-term memory
        self.memory.delete_memory(Literal(memory_id, datatype=XSD.integer))

    def print_memory(self) -> None:
        """
        Print the current memory system (short-term and long-term memories).
        """
        print("\nMemory System:")
        self.memory.print_memories()

    def clear_short_term_memories(self) -> None:
        """
        Clear all remaining short-term memories.
        """
        self.memory.clear_short_term_memories()

/home/tk/.virtualenvs/humemai/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


## Run simple conversation

In [2]:

# Initialize the Agent
agent = Agent()

# First conversation with the agent
agent.process_input("Alice met Bob in Paris on May 5th, 2023 during the AI Conference.")
agent.print_memory()

# Clear remaining short-term memories
agent.clear_short_term_memories()
agent.print_memory()

# New conversation with the agent
agent.process_input("Charlie saw Alice in Seoul on June 10th, 2024.")
agent.print_memory()

# Clear remaining short-term memories
agent.clear_short_term_memories()
agent.print_memory()

# More conversations with the agent (10 additional sentences)
sentences = [
    "David attended a workshop in London on September 15th, 2021.",
    "Emma spoke with John at the university library in New York last week.",
    "Sophia collaborated with Michael on a project in San Francisco in March 2022.",
    "Lucas visited his grandmother in Chicago during Christmas of 2019.",
    "Mia and Noah went hiking together in the Rocky Mountains in July.",
    "Liam studied computer science at MIT from 2018 to 2022.",
    "Isabella discussed climate change with experts at a conference in Copenhagen in 2020.",
    "Olivia and Ethan traveled to Japan for a business trip in April 2021.",
    "James met William at a restaurant in Berlin two days ago.",
    "Ava gave a speech on AI at a technology summit in Silicon Valley last month.",
]

# Process each sentence and interact with the memory system
for sentence in sentences:
    agent.process_input(sentence)
    agent.print_memory()
    agent.clear_short_term_memories()

# Print final memory state after all sentences
print("\nFinal memory system state after processing all sentences:")
agent.print_memory()


Entities found: [('Alice', 'PERSON'), ('Bob', 'PERSON'), ('Paris', 'GPE'), ('May 5th, 2023', 'DATE'), ('the AI Conference', 'FAC')]
Extracted triples: [(rdflib.term.URIRef('https://example.org/Alice'), rdflib.term.URIRef('https://example.org/meet'), rdflib.term.URIRef('https://example.org/Bob')), (rdflib.term.URIRef('https://example.org/Bob'), rdflib.term.URIRef('https://example.org/meet'), rdflib.term.URIRef('https://example.org/Paris'))]
Location: Paris
Time: 2023-05-05T00:00:00

No long-term memories available. Retrieving all memories.
Moving memory to semantic long-term with strength: 5, derivedFrom: Observation
Moving memory to semantic long-term with strength: 6, derivedFrom: Research

Memory System:
(Alice, meet, Bob, {'memoryID': '0', 'recalled': '0', 'knownSince': '2024-10-24T14:44:28.184148', 'strength': '5', 'derivedFrom': 'Observation'})
(Bob, meet, Paris, {'memoryID': '1', 'recalled': '0', 'knownSince': '2024-10-24T14:44:28.184454', 'strength': '6', 'derivedFrom': 'Resear